In [ ]:
# python
import os
from functools import partial
# pytorch
import torch
from torch import nn
# pytorch utilidades
from torch.utils.data import random_split
# ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
# yo
from cargar_datos import cargar_datasets, cargar_dataloaders
from modelo_convolucional import instanciarRed


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0):
    data_dir = os.path.abspath("./data")
    checkpoint_dir = os.path.abspath("./checkpoint_dir")

    # se carga una vez, para que si no esta descargado, se descarge y se valide, el resto de veces en las pruebas, se saltara esta parte.
    cargar_datasets(data_dir)
    """
    variable de configuracion, Ray tune modificara sus valores en el proceso de ajuste de hiperparametros.
    """
    config = {
        "batch_size": tune.choice([8, 16, 32]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        # red convolucional
        "cant_filtros_conv1": tune.choice([6, 12, 18]),
        "kernel_size_maxpool1": tune.choice([2, 3]),
        "cant_filtros_conv2": tune.choice([16, 22, 28]),
        "kernel_size_maxpool2": tune.choice([2, 3]),
        "full_l1": tune.choice([120, 140, 160]),
        "full_l2": tune.choice([84, 104, 124])
    }
    """
    configurar Ray y correr las pruebas en busqueda de hiperparametros
    """
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=list(config.keys()),
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        # partial es una funcion de orden superior de tipo curry
        partial(train_cifar, data_dir=data_dir, checkpoint_dir=checkpoint_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    """
    Validar el desempeño de la red con mejor puntuacion en la busqueda de hiperparametros contra el conjunto de datos de validacion.
    """
    best_trained_model = instanciarRed(config)
    device = getDevice()
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device, data_dir)
    print("Best trial test set accuracy: {}".format(test_acc))


def test_accuracy(model, device="cpu", data_dir="./data"):
    _, testset = cargar_datasets(data_dir)
    testloader = cargar_dataloaders(testset, 4)

    correct = 0
    total = 0
    # pone a la red en moodo evaluacion, desactiva capas dropout.
    model.eval()
    # desactiva el proceso de calculo y guardado de valores intermedios
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


def getDevice():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            raise Exception(
                "la pc cuenta con multiples gpus, deberia utilizar DataParallel")
            #model = nn.DataParallel(model)
    return device


def train_cifar(config, checkpoint_dir=None, data_dir=None):
    model = instanciarRed(config)

    device = getDevice()
    # le dice a Pytorch donde debe ejecutar el modelo.
    model.to(device)
    # CrossEntropyLoss requiere el output "logits", no es necesario pasarlo por el softmax, ya que lo calcula dentro. https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    criterion = nn.CrossEntropyLoss()

    # se necesita enviar los parametros del modelo al optimizador para que los pueda actualizar.
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["learning_rate"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, _ = cargar_datasets(data_dir)
    # reservar una parte de los datos de entrenamiento para validacion.
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = cargar_dataloaders(train_subset, config["batch_size"])
    valloader = cargar_dataloaders(val_subset, config["batch_size"])

    for epoch in range(10):  # loop over the dataset multiple times

        train_loop(model, device, trainloader, criterion, optimizer, epoch)
        loss, accuracy = validation_loss(model, device, valloader, criterion)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)
    print("Finished Training")


def train_loop(model, device, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_steps += 1
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0


def validation_loss(net, device, valloader, criterion):
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    net.eval()
    for i, data in enumerate(valloader, 0):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    loss = val_loss / val_steps
    accuracy = correct / total
    return loss, accuracy


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=1, max_num_epochs=10, gpus_per_trial=0.5)
